In [ ]:
# WARNING: advised to install a specific version, e.g. qrules==0.1.2
%pip install -q qrules[doc,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Combining reactions

This notebooks shows how to combine two {class}`.ReactionInfo` objects that describe transitions for one equivalent initial-to-final state reaction. In this example, we have one {class}`.ReactionInfo` object that is generated all selection rules for the strong force and one that is generated for with {ref}`specific interaction types per node <usage/reaction#Select interaction types>`.

In [ ]:
import qrules

initial_state = [("J/psi(1S)", [-1, +1])]
final_state = ["eta", "p", "p~"]
reaction = qrules.generate_transitions(
    initial_state,
    final_state,
    allowed_intermediate_particles=["N"],
    allowed_interaction_types="strong",
)

In [ ]:
import graphviz

src = qrules.io.asdot(reaction.transitions, collapse_graphs=True)
graphviz.Source(src)

In [ ]:
from qrules.particle import Particle

PDG = qrules.load_pdg()
X1835 = Particle(
    # https://pdglive.lbl.gov/Particle.action?init=0&node=M085&home=MXXX005
    name="X(1835)",
    pid=9999991,
    mass=1.8265,
    width=0.242,
    spin=0,
    parity=-1,
    c_parity=+1,
)
PDG.add(X1835)
reaction_x1835 = qrules.generate_transitions(
    initial_state,
    final_state,
    allowed_intermediate_particles=[X1835.name],
    particle_db=PDG,
)

In [ ]:
import graphviz

src = qrules.io.asdot(reaction_x1835.transitions, collapse_graphs=True)
graphviz.Source(src)

In [ ]:
from collections import defaultdict

from qrules.transition import ReactionInfo, StateTransitionCollection


def merge_reactions(*reactions: ReactionInfo) -> ReactionInfo:
    old_groups = []
    for reaction in reactions:
        old_groups.extend(reaction.transition_groups)
    merged_transitions = defaultdict(list)
    for group in old_groups:
        merged_transitions[group.topology].extend(group.transitions)
    transition_groups = {
        topology: StateTransitionCollection(transitions)
        for topology, transitions in merged_transitions.items()
    }
    return ReactionInfo(transition_groups.values(), formalism=reaction.formalism)


merged_reaction = merge_reactions(reaction, reaction_x1835)

In [ ]:
import graphviz

src = qrules.io.asdot(merged_reaction.transitions, collapse_graphs=True)
graphviz.Source(src)